In [80]:
# requirements 
%pip install pandas --quiet 
%pip install flask requests --quiet
%pip install ollama --quiet

print("Requirements installed successfully!")


You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Requirements installed successfully!


In [81]:
# imports
print("Starting imports...")
import pandas as pd
from flask import Flask, request, jsonify
import ollama
import json
import requests
import re

print("Successfully imported!")

Starting imports...
Successfully imported!


In [82]:
# Load the dataset into a df
carsDF = pd.read_csv('vehicles.csv')

In [83]:
# # view dataset 
# carsDF.head()

Data Cleaning and Preprocessing 

In [84]:
# Normalize text values to title case for consistency
carsDF['Make'] = carsDF['Make'].str.strip().str.upper()  # Remove extra spaces and convert to uppercase
carsDF['Model'] = carsDF['Model'].str.strip().str.upper()  # Remove extra spaces and convert to uppercase


In [85]:
# # Verify normalization
# print(carsDF[['Make', 'Model']])

In [86]:
# Data Type Correction
# Convert numerical columns
carsDF['Year'] = carsDF['Year'].astype(int)
carsDF['SellingPrice'] = carsDF['SellingPrice'].astype(float)
carsDF['Doors'] = carsDF['Doors'].astype(int)


In [87]:
carsDF['PriceRange'] = pd.cut(carsDF['SellingPrice'], 
                              bins=[0, 10000, 20000, 30000, 50000, 100000],
                              labels=['Under 10K', '10K-20K', '20K-30K', '30K-50K', 'Above 50K'])


In [97]:
def extract_preferences(message):
    """
    Function to extract user preferences from a message.
    Arguments:
    - message: User's message as a string.

    Returns:
    - preferences: A dictionary with extracted preferences.
    """
    preferences = {}

    if "new" in message.lower() or "used" in message.lower():
        preferences['car_type'] = 'New' if 'new' in message.lower() else 'Used'
    
    if 'year' in message.lower():
        year = [int(word) for word in message.split() if word.isdigit() and len(word) == 4]
        if year:
            preferences['year'] = year[0]

    if any(make in message.lower() for make in carsDF['Make'].str.lower().unique()):
        preferences['make'] = next(make.title() for make in carsDF['Make'].str.lower().unique() if make in message.lower())

    # Extract price range (e.g., "under $20,000", "between $20,000 and $30,000")
    price_under = re.search(r'under\s*\$?(\d{1,3}(?:,\d{3})*)', message, re.IGNORECASE)
    price_between = re.search(r'between\s*\$?(\d{1,3}(?:,\d{3})*)\s*and\s*\$?(\d{1,3}(?:,\d{3})*)', message, re.IGNORECASE)
    
    if price_under:
        max_price = int(price_under.group(1).replace(',', ''))
        preferences['max_price'] = max_price
    elif price_between:
        min_price = int(price_between.group(1).replace(',', ''))
        max_price = int(price_between.group(2).replace(',', ''))
        preferences['min_price'] = min_price
        preferences['max_price'] = max_price

    # Extract mileage (e.g., "low mileage" or "<50,000 miles")
    if "low mileage" in message.lower():
        preferences['max_miles'] = 50000  # Assume "low mileage" means under 50,000 miles
    mileage = re.search(r'(\d{1,3}(?:,\d{3})*)\s*miles', message, re.IGNORECASE)
    if mileage:
        preferences['max_miles'] = int(mileage.group(1).replace(',', ''))

    # Extract exterior color preference
    colors = carsDF['Ext_Color_Generic'].dropna().apply(str).str.lower().unique()
    for color in colors:
        if color in message.lower():
            preferences['exterior_color'] = color.title()
            break

    # Extract drivetrain preference (e.g., AWD, FWD, RWD)
    drivetrains = ['AWD', 'FWD', 'RWD', '4WD']
    for drivetrain in drivetrains:
        if drivetrain.lower() in message.lower():
            preferences['drivetrain'] = drivetrain.upper()
            break
    

    return preferences


In [107]:
def get_matching_cars(message):
    """
    Function to extract user preferences from a message and return matching cars from the dataset.
    
    Arguments:
    - message: User's message as a string.
    
    Returns:
    - matching_cars: A DataFrame with cars that match user preferences.
    """
    # Extract preferences using the existing function
    preferences = extract_preferences(message)
    
    # Filter the dataset based on preferences
    filtered_cars = carsDF
    
    # Apply each preference as a filter
    if 'car_type' in preferences:
        if preferences['car_type'] == 'New':
            filtered_cars = filtered_cars[filtered_cars['Type'] == 'New']
        elif preferences['car_type'] == 'Used':
            filtered_cars = filtered_cars[filtered_cars['Type'] == 'Used']
    
    if 'year' in preferences:
        filtered_cars = filtered_cars[filtered_cars['Year'] == preferences['year']]
    
    if 'make' in preferences:
        filtered_cars = filtered_cars[filtered_cars['Make'].str.upper() == preferences['make'].upper()]
    
    if 'min_price' in preferences and 'max_price' in preferences:
        filtered_cars = filtered_cars[
            (filtered_cars['SellingPrice'] >= preferences['min_price']) & 
            (filtered_cars['SellingPrice'] <= preferences['max_price'])
        ]
    elif 'max_price' in preferences:
        filtered_cars = filtered_cars[filtered_cars['SellingPrice'] <= preferences['max_price']]
    
    if 'max_miles' in preferences:
        filtered_cars = filtered_cars[filtered_cars['Miles'] <= preferences['max_miles']]
    
    if 'exterior_color' in preferences:
        filtered_cars = filtered_cars[filtered_cars['Ext_Color_Generic'].str.lower() == preferences['exterior_color'].lower()]
    
    if 'drivetrain' in preferences:
        filtered_cars = filtered_cars[filtered_cars['Drivetrain'].str.upper() == preferences['drivetrain'].upper()]
    
    # Return the filtered dataset with relevant columns
    if not filtered_cars.empty:
        return filtered_cars[['Type', 'Stock', 'VIN', 'Year', 'Make', 'Model', 'Body', 'ModelNumber',
       'Doors', 'ExteriorColor', 'InteriorColor', 'EngineCylinders',
       'EngineDisplacement', 'Transmission', 'Miles', 'SellingPrice', 'MSRP',
       'BookValue', 'Invoice', 'Certified', 'Options', 'Style_Description',
       'Ext_Color_Generic', 'Ext_Color_Code', 'Int_Color_Generic',
       'Int_Color_Code', 'Int_Upholstery', 'Engine_Block_Type',
       'Engine_Aspiration_Type', 'Engine_Description', 'Transmission_Speed',
       'Transmission_Description', 'Drivetrain', 'Fuel_Type', 'CityMPG',
       'HighwayMPG', 'EPAClassification', 'Wheelbase_Code', 'Internet_Price',
       'MarketClass', 'PassengerCapacity', 'ExtColorHexCode',
       'IntColorHexCode', 'EngineDisplacementCubicInches', 'PriceRange']].head(5)  # Returning top 10 matches for brevity
    else:
        return None


In [102]:
print(carsDF.columns)


Index(['Type', 'Stock', 'VIN', 'Year', 'Make', 'Model', 'Body', 'ModelNumber',
       'Doors', 'ExteriorColor', 'InteriorColor', 'EngineCylinders',
       'EngineDisplacement', 'Transmission', 'Miles', 'SellingPrice', 'MSRP',
       'BookValue', 'Invoice', 'Certified', 'Options', 'Style_Description',
       'Ext_Color_Generic', 'Ext_Color_Code', 'Int_Color_Generic',
       'Int_Color_Code', 'Int_Upholstery', 'Engine_Block_Type',
       'Engine_Aspiration_Type', 'Engine_Description', 'Transmission_Speed',
       'Transmission_Description', 'Drivetrain', 'Fuel_Type', 'CityMPG',
       'HighwayMPG', 'EPAClassification', 'Wheelbase_Code', 'Internet_Price',
       'MarketClass', 'PassengerCapacity', 'ExtColorHexCode',
       'IntColorHexCode', 'EngineDisplacementCubicInches', 'PriceRange'],
      dtype='object')


In [112]:
# Sample message to test the get_matching_cars function
#test_message = "I'm looking for a used car from 2019, preferably under $30,000 with low mileage."
test_message = "I'm looking for a 2014 Honda with high mileage."

# Get matching cars
matching_cars = get_matching_cars(test_message)

# Display the result
if matching_cars is not None:
    print("Matching Cars:")
    print(matching_cars)
else:
    print("No cars match the specified preferences.")



Matching Cars:
    Type    Stock                VIN  Year   Make            Model  \
8    New  H254838  5CTG8SJWZC5HKDNLB  2018  HONDA             HR-V   
28   New  T965130  Z8JS24DT5ZT69T5R5  2023  HONDA            CIVIC   
34  Used  H108383  CMW5BPVEUF98K6H8D  2014  HONDA           ACCORD   
35  Used  P541001  2UJ6TY3LFP3222GJP  2024  HONDA  CIVIC HATCHBACK   
39  Used  F884425  VRB2D001UP7F84Z4H  2024  HONDA  CIVIC HATCHBACK   

             Body  ModelNumber  Doors              ExteriorColor  ...  \
8   Sport Utility  HOHR-VH558X      4      Lunar Silver Metallic  ...   
28        4dr Car  HOCI-ML043L      4                 Rallye Red  ...   
34        4dr Car  HOAC-FD381Y      4  Alabaster Silver Metallic  ...   
35      Hatchback  HOCI-XT098D      4       Platinum White Pearl  ...   
39      Hatchback  HOCI-AN230U      4       Platinum White Pearl  ...   

   HighwayMPG     EPAClassification Wheelbase_Code Internet_Price  \
8        31.0  Small Station Wagons          102.8      

In [89]:
@app.route('/follow_up', methods=['POST'])
def follow_up():
    conversation_id = request.json.get('conversation_id')

    response = ollama.chat(
        model="your-llama-model",
        message="Can you provide more details like your budget or preferred car features?",
        conversation_id=conversation_id
    )

    return jsonify({"reply": response['content'], "conversation_id": response['conversation_id']})


In [90]:
def search_cars(carsDF, car_type=None, year=None, make=None, model=None, model_number=None, stock=None, vin=None, certified=None):
    """
    Function to filter the cars DataFrame based on user-provided criteria.
    Arguments:
    - df: DataFrame containing car details.
    - car_type: 'New' or 'Used'
    - year: Year of manufacture (e.g., 2020)
    - make: Make of the car (e.g., 'Mazda')
    - model: Model of the car (e.g., 'CX-9')
    - model_number: Internal model code
    - stock: Dealer stock number
    - vin: Vehicle Identification Number
    - certified: Pre-owned certification status (True/False)
    
    Returns:
    - Filtered DataFrame based on provided criteria.
    """
    filtered_df = carsDF.copy()

    if car_type:
        filtered_df = filtered_df[filtered_df['Type'].str.upper() == car_type.upper()]
    if year:
        filtered_df = filtered_df[filtered_df['Year'] == year]
    if make:
        filtered_df = filtered_df[filtered_df['Make'].str.upper() == make.upper()]
    if model:
        filtered_df = filtered_df[filtered_df['Model'].str.upper() == model.upper()]
    if model_number:
        filtered_df = filtered_df[filtered_df['ModelNumber'].str.upper() == model_number.upper()]
    if stock:
        filtered_df = filtered_df[filtered_df['Stock'].str.upper() == stock.upper()]
    if vin:
        filtered_df = filtered_df[filtered_df['VIN'].str.upper() == vin.upper()]
    if certified is not None:  # Certified can be True or False
        filtered_df = filtered_df[filtered_df['Certified'] == certified]

    return filtered_df



In [91]:
def format_car_suggestions(cars):
    suggestions = ""
    for idx, car in cars.iterrows():
        suggestions += f"- {car['Year']} {car['Make']} {car['Model']}, priced at ${car['SellingPrice']:,.0f}. It has {car['Miles']} miles.\n"
    return f"Here are some options I found for you:\n{suggestions}Would you like to learn more about any of these cars?"


In [92]:
def get_car_details(car_id):
    car = carsDF[carsDF['VIN'] == car_id].iloc[0]
    return (f"This is a {car['Year']} {car['Make']} {car['Model']} with {car['Miles']} miles, "
            f"available for ${car['SellingPrice']:,.0f}. It comes with {car['Int_Upholstery']} upholstery "
            f"and has a {car['Fuel_Type']} engine.")


In [93]:
def compare_cars(vin1, vin2):
    car1 = carsDF[carsDF['VIN'] == vin1].iloc[0]
    car2 = carsDF[carsDF['VIN'] == vin2].iloc[0]
    
    comparison = (f"Comparing {car1['Make']} {car1['Model']} vs {car2['Make']} {car2['Model']}:\n"
                  f"- {car1['Make']} {car1['Model']} is priced at ${car1['SellingPrice']:,.0f} with {car1['Miles']} miles.\n"
                  f"- {car2['Make']} {car2['Model']} is priced at ${car2['SellingPrice']:,.0f} with {car2['Miles']} miles.\n"
                  f"{'The first car is more affordable.' if car1['SellingPrice'] < car2['SellingPrice'] else 'The second car is more affordable.'}")
    
    return comparison


In [94]:
def chat_with_llama2(user_message, conversation_id):
    url = "http://localhost:11434/api/chat"
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "llama2",
        "message": user_message,
        "conversation_id": conversation_id
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        response_data = response.json()
        return response_data.get('content', "No response available")
    else:
        return f"Error: {response.status_code} - {response.text}"

In [95]:
# Flask App Definition
app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    user_message = request.json.get('message')
    conversation_id = request.json.get('conversation_id')

    # Get response from Ollama model
    response_content = chat_with_llama2(user_message, conversation_id)

    # Use LLM to extract user preferences
    user_preferences = extract_preferences(response_content)
    filtered_results = search_cars(carsDF, **user_preferences)

    if filtered_results.empty:
        follow_up_message = "I couldn't find any cars matching your preferences. Would you like to modify your criteria?"
    else:
        top_cars = filtered_results.head(5)  # Get top 5 matches
        follow_up_message = format_car_suggestions(top_cars)

    # Respond with Ollama's response and filtered car suggestions
    return jsonify({"reply": follow_up_message, "conversation_id": conversation_id})

# Run the server
if __name__ == "__main__":
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
